<a href="https://colab.research.google.com/github/Pepcoders/Data-Science-February/blob/main/Pytorch/Pytorch_Intro_Feb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
print(torch.__version__)

1.11.0+cu113


In [3]:
a = torch.empty(6,3)
print(a)

tensor([[1.5997e-35, 0.0000e+00, 7.0065e-44],
        [7.0065e-44, 6.3058e-44, 6.7262e-44],
        [7.7071e-44, 6.3058e-44, 6.7262e-44],
        [7.7071e-44, 1.1771e-43, 6.8664e-44],
        [7.5670e-44, 8.1275e-44, 6.7262e-44],
        [7.9874e-44, 8.1275e-44, 7.4269e-44]])


In [4]:
a = torch.rand(4,3)
print(a)

tensor([[0.5062, 0.9293, 0.7298],
        [0.0846, 0.9877, 0.9808],
        [0.6048, 0.0657, 0.0279],
        [0.1556, 0.3904, 0.5784]])


In [5]:
a = torch.zeros(4,3, dtype=torch.long)
print(a)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [6]:
a = torch.tensor([7.8, 5])
type(a)

torch.Tensor

In [7]:
a = a.new_ones(6,5, dtype=torch.double)    # new methods take in sizes
print(a)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]], dtype=torch.float64)


In [8]:
print(a.size())

torch.Size([6, 5])


In [11]:
b = torch.rand(6,5)
print(a + b)

tensor([[1.2846, 1.0933, 1.6736, 1.2029, 1.2331],
        [1.0124, 1.5622, 1.1152, 1.8555, 1.9584],
        [1.3845, 1.8683, 1.5206, 1.3508, 1.6470],
        [1.1222, 1.3658, 1.6795, 1.9185, 1.7592],
        [1.3282, 1.4650, 1.8696, 1.7739, 1.1535],
        [1.6209, 1.8669, 1.8465, 1.8653, 1.0341]], dtype=torch.float64)


In [12]:
print(torch.add(a,b))

tensor([[1.2846, 1.0933, 1.6736, 1.2029, 1.2331],
        [1.0124, 1.5622, 1.1152, 1.8555, 1.9584],
        [1.3845, 1.8683, 1.5206, 1.3508, 1.6470],
        [1.1222, 1.3658, 1.6795, 1.9185, 1.7592],
        [1.3282, 1.4650, 1.8696, 1.7739, 1.1535],
        [1.6209, 1.8669, 1.8465, 1.8653, 1.0341]], dtype=torch.float64)


In [13]:
x = torch.ones(4)
print(x)

tensor([1., 1., 1., 1.])


In [14]:
y = x.numpy()
print(y)

[1. 1. 1. 1.]


In [17]:
import numpy as np
f = np.ones(4)
g = torch.from_numpy(f)
np.add(f, 4-7, out=f)
print(f)
print(g)

[-2. -2. -2. -2.]
tensor([-2., -2., -2., -2.], dtype=torch.float64)


In [26]:
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to("cuda")                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))

tensor([2., 2., 2., 2.], device='cuda:0')
tensor([2., 2., 2., 2.], dtype=torch.float64)


In [18]:
torch.cuda.is_available()

True

The key step is between the last convolution and the first Linear block. Conv2d outputs a tensor of shape [batch_size, n_features_conv, height, width] whereas Linear expects [batch_size, n_features_lin]. To make the two align you need to "stack" the 3 dimensions [n_features_conv, height, width] into one [n_features_lin]. As follows, it must be that n_features_lin == n_features_conv height width. In the original code this "stacking" is achieved by

x = x.view(-1, self.num_flat_features(x))
and if you inspect num_flat_features it just computes this n_features_conv height width product. In other words, your first conv must have num_flat_features(x) input features, where x is the tensor retrieved from the preceding convolution. But we need to calculate this value ahead of time, so that we can initialize the network in the first place...

The calculation follows from inspecting the operations one by one.

input is 32x32 we do a 5x5 convolution without padding, so we lose 2 pixels at each side, we drop down to 28x28 we do maxpooling with receptive field of 2x2, we cut each dimension by half, down to 14x14 we do another 5x5 convolution without padding, we drop down to 10x10 we do another maxpooling, we drop down to 5x5 and this 5x5 is why in the tutorial you see self.fc1 = nn.Linear(16 5 5, 120). It's n_features_conv height width, when starting from a 32x32 image. If you want to have a different input size, you have to redo the above calculation and adjust your first Linear layer accordingly.

For the further operations, it's just a chain of matrix multiplications (that's what Linear does). So the only rule is that the n_features_out of previous Linear matches n_features_in of the next one. Values 120 and 84 are entirely arbitrary, though they were probably chosen by the author such that the resulting network performs well.

In [27]:
#Lenet-5

import torch
import torch.nn as nn
import torch.nn.functional as F


class network(nn.Module):

    def __init__(self):
        super(network, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = mx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = network()
print(net)

network(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
list(net.parameters())[0]

In [31]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [32]:
inp = torch.randn(1, 1, 32, 32)
out = net(inp)
print(out)

tensor([[-0.0571, -0.0353,  0.0850,  0.0342, -0.0136,  0.0847,  0.0872, -0.0356,
          0.0298, -0.0276]], grad_fn=<AddmmBackward0>)


In [33]:
output = net(inp)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6927, grad_fn=<MseLossBackward0>)


In [35]:
torch.randn(1, 1, 32, 32)

tensor([[[[-0.6999, -0.3002, -0.7185,  ..., -2.1788,  0.0079, -0.4762],
          [-0.3074, -1.2085,  1.6936,  ...,  1.0781, -0.9154, -1.8044],
          [-1.0755,  0.3874,  1.6203,  ..., -1.0567,  0.0028,  0.9898],
          ...,
          [ 0.5869, -2.0913,  0.3703,  ..., -0.8713,  1.6704, -1.5693],
          [-0.5363,  1.1995,  0.2379,  ..., -0.3479,  0.2229, -1.0608],
          [ 0.3098,  0.0090,  0.3040,  ...,  1.1471,  1.6158, -0.4676]]]])